<a href="https://colab.research.google.com/github/uhdang/mlim_group_3_final_assignment/blob/main/project/train_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:

- Downsample training set (reduce method to delete irrelevant rows)
- maybe PCA()

# Google Colab Runtime
Run if using Google Colab

In [1]:
# # # import pandas as pd
# # # 
# # import numpy as np# 
# # 
# import o# s# 
# 
# from google.colab import dri# v# e# 
# 
# import # s# y# s#
# drive.mount('/content/dri# v# e# '# )
# sys.path.append('/content/drive/My Drive/My Dri# v# e# '# )
#
# from dataloader import Dat# a# l# o# ader
# # L# o# a# d#  Data
# path = "/content/drive/MyDrive/My Drive/"

# Local Runtime
Run if using notebook locally

In [6]:
import pandas as pd
import os
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

from dataloader import Dataloader
from dataloader import create_combined_dict

# Load Data
path = os.getcwd() + "/../data/"

# Load Data

In [7]:
# load data which creates baskets and coupons
data = Dataloader(path)

In [8]:
# Create Categories before CV as they don't change and consume time otherwise
data.create_category_table(10000)

In [10]:
%%capture
weeks = [86, 87, 88, 89]
shopper_list = list(range(2000))
shopper_chunks = [shopper_list[i:i + 100] for i in range(0, len(shopper_list), 100)]

cv_dict = {
    'X_train': list(),
    'y_train': list(),
    'X_test': list(),
    'y_test': list()
}

In [11]:
for i, week in enumerate(weeks):
  print(f"week: {week}")

  X_train_list = list()
  y_train_list = list()
  X_test_list = list()
  y_test_list = list()

  for idx, shopper in enumerate(tqdm(shopper_chunks)):
      # print(f"shopper_chunk index: {idx}")

      # train-test-split
      data.train_test_split(week, shopper)

      # data add categories
      data.add_categories()

      # create features
      data.create_feature_dict()

      # combine everything
      if i == 0:
          X_train, y_train, X_test, y_test = data.make_featured_data()
          X_train_list.append(X_train)
          y_train_list.append(y_train)
          del X_train, y_train
      else:
          _, _, X_test, y_test = data.make_featured_data()
          X_test_list.append(X_test)
          y_test_list.append(y_test)
          del X_test, y_test

  cv_dict = create_combined_dict(X_train_list, y_train_list, X_test_list, y_test_list, cv_dict)

  5%|▌         | 1/20 [03:24<1:04:39, 204.18s/it]


week: 86


KeyboardInterrupt: 

In [ ]:
cv_dict["X_train"][0]["shopper"]

In [ ]:
import pickle

with open('cv_dict.pickle', 'wb') as handle:
    pickle.dump(cv_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Hyperparameter Tuning

In [7]:
%%capture
!pip install optuna

In [ ]:
import optuna
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import roc_auc_score

In [9]:
# 1. Define an objective function to be maximized.
class Objective(object):
  def __init__(self, data):
    self.data = data
  def __call__(self, trial):
    # 2. Suggest values for the hyperparameters using a trial object.
    params = {
        # 'objective': 'binary',
        # 'metric': 'binary_logloss',
        # 'verbosity': -1,
        # 'boosting_type': 'gbdt',
        # 'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        # 'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        # 'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        # 'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        # 'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        # 'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }

    lgb_clf = LGBMClassifier(**params)
    auc_dict = {}
    folds = len(self.data['X_train'])

    for fold in range(folds):
      lgb_clf.fit(
          X = self.data['X_train'][fold], 
          y = self.data['y_train'][fold]
      )
      auc_score = roc_auc_score(
          y_true = self.data['y_test'][fold],
          y_score = lgb_clf.predict_proba(self.data['X_test'][fold])[:,1]
      )
      auc_dict[f'Fold {str(fold)}'] = auc_score

    accuracy = sum(auc_dict.values()) / len(auc_dict.values())

    return accuracy    

In [12]:
# 3. Create a study object and optimize the objective function.

# Load the dataset in advance for reusing it each trial execution.
objective = Objective(cv_dict)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)

[I 2021-03-18 20:22:19,213] A new study created in memory with name: no-name-93f45318-8e1b-4824-afa1-1da842bdf9a5
[I 2021-03-18 20:23:18,060] Trial 0 finished with value: 0.9439790932029444 and parameters: {'min_child_samples': 25}. Best is trial 0 with value: 0.9439790932029444.
[I 2021-03-18 20:24:15,686] Trial 1 finished with value: 0.9459737152065653 and parameters: {'min_child_samples': 40}. Best is trial 1 with value: 0.9459737152065653.


# Prediction and Revenue Optimization

# Playground

In [ ]:
# train-test-split
data.train_test_split(89, list(range(50)))

# data add categories 
data.add_categories()
data.create_feature_dict()
X_train = data._make_full_table(data.baskets_train, data.coupons_train)
X_test = data._make_full_table(data.baskets_test, data.coupons_test)

X_train = data._merge_features(X_train, data.feature_dict)
X_test = data._merge_features(X_test, data.feature_dict)

In [6]:
X_train.head()

,week,shopper,product,price,target,category,discount,coupon,original_price,ratio_product_count,ratio_category_count,reordered_product,reordered_category,coupon_in_same_category,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper
0,0,0,0,688.0,0.0,12,0.0,No,688,0.000000,0.089888,0.0,1.0,Yes,0.722222,0.960000,587.203947,8.539326,54,25
1,0,1,0,688.0,0.0,12,0.0,No,688,0.067416,0.314607,1.0,1.0,Yes,0.830986,1.000000,584.065350,7.393258,71,24
2,0,2,0,688.0,0.0,12,0.0,No,688,0.000000,0.595506,0.0,1.0,No,0.700000,0.954545,571.918919,5.820225,60,22
3,0,3,0,688.0,0.0,12,0.0,No,688,0.000000,0.415730,0.0,1.0,No,0.714286,1.000000,561.846361,8.337079,91,24
4,0,4,0,688.0,0.0,12,0.0,No,688,0.000000,0.089888,0.0,1.0,No,0.764706,1.000000,583.750455,6.168539,34,20
